In [1]:
from datasets import load_dataset
import ollama
from tqdm.notebook import tqdm
import pandas as pd

def getResponse(inputString):
    response = ollama.chat(model='llama3.1', messages=[
        {
            'role': 'user',
            'content': inputString,
        }],
        options={'temperature':1.0 }
    )
    return response['message']['content']

ds = load_dataset("SetFit/tweet_sentiment_extraction")

answerConversion = {'P': 'positive', 'I': 'neutral', 'N': 'negative'}
acronymConversions = {'positive':'P', 'neutral':'I', 'negative':'N'}

def parseOutput(modelOutput):
    if (modelOutput[0] in ['P', 'I', 'N']) and (modelOutput[1:3] == ' -'):
        return answerConversion[modelOutput[0]]
    else:
        return -1
    


from collections import defaultdict
ds = load_dataset("SetFit/tweet_sentiment_extraction")
def get_first_n_per_label(dataset, n=3):
    label_dict = defaultdict(list)  
    

    for sample in dataset:
        label = sample['label']
        if len(label_dict[label]) < n:
            label_dict[label].append(sample)
        if all(len(v) >= n for v in label_dict.values()):
            break

    return label_dict


NUM_SAMPLES = 20
first_3_per_label = get_first_n_per_label(ds['train'], n=NUM_SAMPLES)

order = [2, 1, 0]  
interleaved_list = []
for i in range(NUM_SAMPLES):  
    for label in order:
        interleaved_list.append(first_3_per_label[label][i])

for sample in interleaved_list:
    print(sample['text'], sample['label_text'])







Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


2am feedings for the baby are fun when he is all smiles and coos positive
 I`d have responded, if I were going neutral
 Sooo SAD I will miss you here in San Diego!!! negative
 Journey!? Wow... u just became cooler.  hehe... (is that possible!?) positive
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth neutral
my boss is bullying me... negative
I really really like the song Love Story by Taylor Swift positive
Soooo high neutral
 what interview! leave me alone negative
Playing Ghost Online is really interesting. The new updates are Kirin pet and Metamorph for third job.  Can`t wait to have a dragon pet positive
 Both of you neutral
 Sons of ****, why couldn`t they put them on the releases we already bought negative
the free fillin` app on my ipod is fun, im addicted positive
 as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff neutral
My Sharpie is running DANGERously low on ink negative
ju

In [2]:
print(len(interleaved_list))
featureEngineering = f'''[Instructions]: You are a sentiment analyzer tasked with classifying tweets. Given a question, compute the sentiment as P for Positive, I for neutral (indifferent) and N for negative. Give exactly one classification.'''

for i in range(len(interleaved_list)):
    featureEngineering = featureEngineering + f"\n[Example {i}] [Text] {interleaved_list[i]['text']} [Classification] {interleaved_list[i]['label_text']}"
featureEngineering = featureEngineering + '''\n[Instructions Repeated]: You are a sentiment analyzer tasked with classifying tweets. Given a question, compute the sentiment as P for Positive, I for neutral (indifferent) and N for negative. Give exactly one classification.
[Format]: Format your answer as: P OR I OR N - EXPLANATION HERE
[Question]: '''


print(featureEngineering)

df = pd.read_csv('trump_harris_tweets.csv')
trumpTweets = list(df['trump'])
harrisTweets = list(df['harris'])

60
[Instructions]: You are a sentiment analyzer tasked with classifying tweets. Given a question, compute the sentiment as P for Positive, I for neutral (indifferent) and N for negative. Give exactly one classification.
[Example 0] [Text] 2am feedings for the baby are fun when he is all smiles and coos [Classification] positive
[Example 1] [Text]  I`d have responded, if I were going [Classification] neutral
[Example 2] [Text]  Sooo SAD I will miss you here in San Diego!!! [Classification] negative
[Example 3] [Text]  Journey!? Wow... u just became cooler.  hehe... (is that possible!?) [Classification] positive
[Example 4] [Text] http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth [Classification] neutral
[Example 5] [Text] my boss is bullying me... [Classification] negative
[Example 6] [Text] I really really like the song Love Story by Taylor Swift [Classification] positive
[Example 7] [Text] Soooo high [Classification] neutral
[Example 8] [Text

In [10]:
trumpOutputs = []

for sample in trumpTweets:
    modelOutput = getResponse(featureEngineering + sample)
    parsedModelOutput = parseOutput(modelOutput)
    trumpOutputs.append([parsedModelOutput, sample, modelOutput])

In [11]:
kamalaOutputs = []

for sample in harrisTweets:
    modelOutput = getResponse(featureEngineering + sample)
    parsedModelOutput = parseOutput(modelOutput)
    kamalaOutputs.append([parsedModelOutput, sample, modelOutput])

In [13]:
for i in trumpOutputs:
    print(i)
print('-'*30)
for i in kamalaOutputs:
    print(i)

['negative', '#Trump is literally asleep at the RNC event. Asleep. #TrumpAssasinationAttempt #Trump2024 #Trump', 'N - The tone of this tweet is critical and mocking, suggesting that Trump\'s lack of engagement or response to an issue (being "asleep") is a problem, implying he may not be fit for office, and making jokes about the idea of assassinating him. This suggests a negative sentiment towards Trump.']
['negative', "So... You didn't know what 45-47 meant? Now you do! #Trump", "N - The tweet is sarcastic and appears to be criticizing Trump's campaign slogans, implying that his messages are not universally understood. The tone is negative."]
['negative', 'Rep. Marjorie Taylor Greene (R-GA.) calls out Rep. Jamie Raskin (D-MD.) and the Democrats after a subcommittee hearing in which Dr. Anthony Fauci testified about the COVID response. Do you agree with Rep. Greene’s statement?(Video via @RepMTG) #OAN #Hearing #MTG #Trump #Floyd', "N - The tone of the tweet suggests that Rep. Greene's 